In [1]:
%pylab inline
import pandas as pd
import itertools
import scipy
import scipy.optimize

Populating the interactive namespace from numpy and matplotlib


## Нижня модель LUAM
<img src="./lower_model.png"/>
## Верхня модель LUAM
<img src="./upper_model.png"/>

In [2]:
A = np.array([
    [[1, 1], [3, 4], [6, 6], [6, 7]],
    [[1/4, 1/3], [1, 1], [3, 4], [3, 4]],
    [[1/6, 1/6], [1/4, 1/3], [1, 1], [3, 4]],
    [[1/7,1/6], [1/4, 1/3], [1/4, 1/3], [1, 1]]
])
A = np.array([
    [[1, 1], [1, 3], [3, 5], [5, 7], [5, 9]],
    [[1/3, 1], [1, 1], [1, 4], [1, 5], [1, 4]],
    [[1/5, 1/3], [1/4, 1], [1, 1], [1/5, 5], [2, 4]],
    [[1/7, 1/5], [1/5, 1], [1/5, 5], [1, 1], [1, 2]],
    [[1/9, 1/5], [1/4, 1], [1/4, 1/2], [1/2, 1], [1, 1]]
])
AL = A[:, :, 0]
AU = A[:, :, 1]

def fill_lower(r, a, k, i, j):
    r[k, j + a.shape[1]] = a[i, j]
    r[k, i] = -1

def fill_upper(r, a, k, i, j):
    r[k,  i + a.shape[0]] = 1
    r[k,  j] = -a[i, j]
        
def create_matrix(a, f):
    " w_1^L, ... w_n^L, w_1^U, ..., w_n^[l]"
    n, _ =  a.shape
    r = np.zeros((n * n, 2 * n))
    for k, p in enumerate(itertools.product(range(n), range(n))):
        if (p[0] != p[1]):
            f(r, a, k, p[0], p[1])
    return r

def third_lower(n):
    r = np.zeros((n, 2 * n))
    for i in range(n):
        r[i, i] = -1
        for j in range(n):
            if i != j:
                r[i, j + n] = -1
    return r

def fourth_lower(n):
    r = np.zeros((n, 2 * n))
    for i in range(n):
        r[i, i + n] = 1
        for j in range(n):
            if i != j:
                r[i, j] = 1
    return r

def fiveth_lower(n):
    r = np.zeros((n, 2 * n))
    for i in range(n):
        r[i, i] = 1
        r[i, i + n] = -1
    return r
        
def solve(l, u, sz=1):
    m, n = l.shape
    s = 2 * n * n
    assert(m == n)
    assert(l.shape == u.shape)
    a1 = sz * create_matrix(l, fill_lower)
    a2 = sz * create_matrix(u, fill_upper)
    a3 = third_lower(n)
    a4 = fourth_lower(n)
    a5 = fiveth_lower(n)
    a_components = [a1, a2, a3, a4, a5]
    a = np.concatenate(a_components)
    b = np.zeros(s + 3 * n)
    b[s:s + n] = -1
    b[s + n: s + 2 * n] = 1
    c = np.zeros(n + n)
    c[0:n] = sz
    c[n:] = -sz
    data = [c, a, b]
    result = scipy.optimize.linprog(
        *data,
        options=dict(disp=True),
        bounds=[(0.0001, np.inf) for i in range(n + n)]
    )
    return data, result

In [3]:
def prepare_report(eq, solution, n):
    columns = ["$w%d^{L}_{%d}$" % (n, i) for i in range(1, eq[0].size // 2 + 1)] + [
        "$w%d^{U}_{%d}$" %  (n, i) for i in range(1, eq[0].size // 2 + 1)] + ["$b$"]
    a = pd.DataFrame(np.hstack([eq[1], eq[2].reshape(eq[2].size, 1)]), columns=columns)
    n = len(columns) - 1
    df_x1 = pd.DataFrame(solution.x[0:n//2].reshape(1, n//2), columns=columns[0:n//2])
    df_x2 = pd.DataFrame(solution.x[n//2:].reshape(1, n//2), columns=columns[n//2:-1])
    return df_x1, df_x2, a

def fuzzy_advantage(a_low, a_mid, a_up):
    a = np.stack((a_low, a_mid, a_up))
    rsi = a.sum(axis=2)
    si = rsi / rsi.sum(axis=1)[::-1, None]
    mm = np.meshgrid(si[1], si[1]) # mm[1] ~ m_i, mm[0] ~ m_j 
    lu = np.meshgrid(si[0], si[2]) # lu[1] ~ u_i, lu[0] ~ l_j
    vss = np.zeros_like(a_low)
    vss[mm[1] >= mm[0]] = 1
    mask_mixed = (mm[1] < mm[0]) * (lu[1] >= lu[0])
    vals_mixed = (lu[1] - lu[0]) / (lu[1] - mm[1] + mm[0] - lu[0])
    vss[mask_mixed] = vals_mixed[mask_mixed]
    print('Matrix S(l,m,u):\n', si.T)
    print('Matrix VS:\n',vss)
    weights = np.amin(vss, axis=1)
    # calculated weights
    return weights / np.linalg.norm(weights, 1)

In [4]:
solution1 = solve(AL, AU, sz=1)
r1 = prepare_report(*solution1, 1)
solution2 = solve(AL, AU, sz=-1)
r2 = prepare_report(*solution2, 2)

Optimization terminated successfully.
         Current function value: -0.223535   
         Iterations: 45
Optimization terminated successfully.
         Current function value: 0.618182    
         Iterations: 23


# Матриця $\begin{bmatrix}A | b\end{bmatrix}$

In [5]:
r1[-1]

,$w1^{L}_{1}$,$w1^{L}_{2}$,$w1^{L}_{3}$,$w1^{L}_{4}$,$w1^{L}_{5}$,$w1^{U}_{1}$,$w1^{U}_{2}$,$w1^{U}_{3}$,$w1^{U}_{4}$,$w1^{U}_{5}$,$b$
0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0
1,-1.000000,0.0,0.0,0.0,0.0,0.000000,1.00,0.00,0.0,0.0,0.0
2,-1.000000,0.0,0.0,0.0,0.0,0.000000,0.00,3.00,0.0,0.0,0.0
3,-1.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,5.0,0.0,0.0
4,-1.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,5.0,0.0
5,0.000000,-1.0,0.0,0.0,0.0,0.333333,0.00,0.00,0.0,0.0,0.0
6,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,0.0,0.0
7,0.000000,-1.0,0.0,0.0,0.0,0.000000,0.00,1.00,0.0,0.0,0.0
8,0.000000,-1.0,0.0,0.0,0.0,0.000000,0.00,0.00,1.0,0.0,0.0
9,0.000000,-1.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.0,1.0,0.0


# Розв'язок нижньої моделі

In [6]:
r1[0]

,$w1^{L}_{1}$,$w1^{L}_{2}$,$w1^{L}_{3}$,$w1^{L}_{4}$,$w1^{L}_{5}$
0,0.422535,0.178101,0.140845,0.076329,0.070423


In [7]:
r1[1]

,$w1^{U}_{1}$,$w1^{U}_{2}$,$w1^{U}_{3}$,$w1^{U}_{4}$,$w1^{U}_{5}$
0,0.534303,0.28169,0.140845,0.084507,0.070423


# Розв'язок верхньої моделі

In [8]:
r2[0]

,$w2^{L}_{1}$,$w2^{L}_{2}$,$w2^{L}_{3}$,$w2^{L}_{4}$,$w2^{L}_{5}$
0,0.290909,0.136364,0.027273,0.036364,0.045455


In [9]:
r2[1]

,$w2^{U}_{1}$,$w2^{U}_{2}$,$w2^{U}_{3}$,$w2^{U}_{4}$,$w2^{U}_{5}$
0,0.409091,0.290909,0.181818,0.136364,0.136364
